**Stacking** is another ensemble method. In stacking, we train some number of level-0 estimators on a subset of the training data, generate predictions on yet-unseen training data (e.g. from a k-fold cross validation), then feed these predictions along with all other training data seen so far to a new set of estimators (level-1 estimators).

In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

In [14]:
# get the data from https://archive.ics.uci.edu/dataset/186/wine+quality
df_red = pd.read_csv('data/winequality-red.csv',sep = ';')
df_red['color'] = 1
df_white = pd.read_csv('data/winequality-white.csv',sep = ';')
df_white['color'] = 0
df = pd.concat([df_red,df_white])
df.index = range(len(df))

By the way, we can save a dataframe to csv.

In [15]:
df.to_csv('wine.csv',index=False) # index=False: does not save the index as a column
# df = pd.read_csv('wine.csv')
# df.head()

Predicting quality.

In [16]:
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'color']
target = ['quality']

x_tr,x_te,y_tr,y_te = train_test_split(df[features],df[target],
                                       test_size = 0.2)
x_tr,x_va,y_tr,y_va = train_test_split(x_tr,y_tr,
                                       test_size = 0.5)

In [17]:
n_folds = 6
kf = KFold(n_splits=n_folds)

# get the cut positions of the kfold split of x_tr
cuts = []
for (idx_big,idx_small) in kf.split(x_tr):
    cuts.append(idx_small[-1]+1)

# insert a cut at 0
cuts.insert(0,0)

In [26]:
cuts

[0, 433, 866, 1299, 1732, 2165, 2598]

In [21]:
trees = [] # initialize a list of trees

# we want to train each tree on a fold along with predictions from previous trees
for i in range(len(cuts)-1):
    tr = DecisionTreeRegressor(random_state = i)
    tr.fit(x_tr.iloc[cuts[i]:cuts[i+1]],y_tr.iloc[cuts[i]:cuts[i+1]])
    x_tr[f'preds_{i}'] = tr.predict(x_tr)
    trees.append(tr)    

In [25]:
pd.concat([x_tr,y_tr],axis=1).head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
5528            6.6              0.32         0.47            15.6      0.063   
4154            6.3              0.13         0.42             1.1      0.043   
1234            7.1              0.75         0.01             2.2      0.059   
6039            6.7              0.34         0.31            16.4      0.051   
4437            6.9              0.15         0.29             2.3      0.033   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
5528                 27.0                 173.0  0.99872  3.18       0.56   
4154                 63.0                 146.0  0.99066  3.13       0.72   
1234                 11.0                  18.0  0.99242  3.39       0.40   
6039                 20.0                 146.0  0.99834  3.06       0.54   
4437                 14.0                  82.0  0.99132  3.10       0.58   

      alcohol  color  preds_0  preds_1  preds_2  preds_3  preds_4  preds_5  \
5528      9.0      0      5.0      6.0      5.0      5.0      5.0      5.0   
4154     11.2      0      7.0      6.0      7.0      7.0      6.0      5.0   
1234     12.8      1      6.0      5.0      6.0      8.0      7.0      6.0   
6039      9.1      0      5.0      6.0      4.0      5.0      5.0      5.0   
4437     11.2      0      7.0      7.0      6.0      7.0      6.0      6.0   

      quality  
5528        5  
4154        7  
1234        6  
6039        5  
4437        7

In [27]:
# make predictions on the validation set
# this involves making the successive predictions for each tree
for i in range(len(cuts)-1):
    x_va[f'preds_{i}'] = trees[i].predict(x_va)

In [28]:
# compute the R^2 of the final model
sse = np.sum((x_va.iloc[:,-1]-y_va['quality'])**2)
sse_null = np.sum((y_va['quality']-np.mean(y_va['quality']))**2)
1-sse/sse_null

-0.26421610075208046

It's pretty bad, but this is to be expected, as our models have the same skill.

In [29]:
# check the number of times we got exactly the right answer
np.sum((x_va.iloc[:,-1]==y_va['quality']))

1172

We can try increasing the number of trees in each level.

In [32]:
df = pd.read_csv('wine.csv')
x_tr,x_te,y_tr,y_te = train_test_split(df[features],df[target],test_size = 0.2)
x_tr,x_va,y_tr,y_va = train_test_split(x_tr,y_tr,test_size = 0.5)
n_folds = 6
kf = KFold(n_splits=n_folds)
cuts = []
for (idx_big,idx_small) in kf.split(x_tr):
    cuts.append(idx_small[-1]+1)
cuts.insert(0,0)

def forest_predict(forest,inpt):
    preds = []
    for tree in forest:
        preds.append(tree.predict(inpt))
    return(sum(preds)/len(preds))

trees = [[],[],[],[],[],[]] # initialize a list of tree levels
level_size = 4 # specify the number of trees in each level

# we want to train each forest on a fold along with predictions from previous forests
for i in range(len(cuts)-1):
    for j in range(level_size):
        tr = DecisionTreeRegressor(random_state = j)
        tr.fit(x_tr.iloc[cuts[i]:cuts[i+1]],y_tr.iloc[cuts[i]:cuts[i+1]])
        trees[i].append(tr)
    x_tr[f'preds_{i}'] = forest_predict(trees[i],x_tr)

In [35]:
pd.concat((x_tr,y_tr),axis=1).head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
3343            6.6             0.220         0.58             1.1      0.133   
5493            5.9             0.435         0.16             6.4      0.031   
1435           10.2             0.540         0.37            15.4      0.214   
2800            7.0             0.170         0.37             1.5      0.028   
4589            5.0             0.270         0.32             4.5      0.032   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
3343                 52.0                 136.0  0.99320  3.10       0.30   
5493                 21.0                 134.0  0.99151  3.24       0.46   
1435                 55.0                  95.0  1.00369  3.18       0.77   
2800                 26.0                  75.0  0.99220  3.30       0.46   
4589                 58.0                 178.0  0.98956  3.45       0.31   

      alcohol  color  preds_0  preds_1  preds_2  preds_3  preds_4  preds_5  \
3343      9.1      0      5.0      5.0     5.75      5.0     6.00     5.00   
5493     12.2      0      6.0      7.5     8.00      7.0     7.00     6.25   
1435      9.0      1      6.0      5.0     6.00      5.5     4.25     5.00   
2800     10.8      0      7.0      7.0     6.00      6.0     6.00     6.00   
4589     12.6      0      7.0      7.0     7.00      6.0     6.00     7.00   

      quality  
3343        5  
5493        6  
1435        6  
2800        7  
4589        7

In [ ]:
# make predictions on the validation set
# this involves making the successive predictions for each tree
for i in range(len(cuts)-1):
    x_va[f'preds_{i}'] = forest_predict(trees[i],x_va)

In [40]:
# make predictions on the validation set
# this involves making the successive predictions for each tree
for i in range(len(cuts)-1):
    x_va[f'preds_{i}'] = forest_predict(trees[i],x_va)

sse = np.sum((x_va.iloc[:,-1]-y_va['quality'])**2)
sse_null = np.sum((y_va['quality']-np.mean(y_va['quality']))**2)
1-sse/sse_null

-0.4585691876324982

In [41]:
sum(x_va.iloc[:,-1]==y_va['quality'])

1110